## Informational
### Show all Maple Syrup Campaigns

In [ ]:
-- CAMPAIGN only maple syrup capaigns
select *
from staging.salesforce.stg_salesforce__campaign
where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
  and is_deleted=false

## Informational
### Show Campaign Members

In [ ]:
select *
from staging.salesforce.stg_salesforce__campaign_member
where is_deleted=false

# Step 1
## Merge Capaign Members with Campaigns

In [ ]:
-- CAPAIGN ALL
with
campaigns as (
    -- still only looking at Maple Syrup campaigns ... for now.
    select 
        id as campaign_id,
        name as campaign_name,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from staging.salesforce.stg_salesforce__campaign
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id        
    from staging.salesforce.stg_salesforce__campaign_member
    where is_deleted=false
),

final as (
    select 
        c.*,
        m.contact_id
    from campaigns c
    inner join members m 
        on m.campaign_id=c.campaign_id
)

select * from final
order by campaign_start_date, contact_id 

In [ ]:
with 
contacts as (
    select *
    from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

accounts as (
    select *
    from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

all_contacts as (
    select 
        c.id as contact_id,
        c.account_id as account_id,
        c.name as contact_name,
        c.contact_status_c as contact_status,
        c.contact_owner_id,
        c.contact_owner_name,
        c.referred_by_c as contact_referring_contact_id,
        r.account_id as contact_referring_account_id,
        r.name as contact_referring_contact_name
    from contacts c
    left join contacts r
        on r.id=c.referred_by_c
),

contacts_add_account as (
    select 
        c.*,
        a.name as account_name,
        case 
            when a1.name=a.name then null else a1.name
        end as contact_referring_account_name
    from all_contacts c
    left join accounts a 
        on a.id=c.account_id
    left join accounts a1
        on a1.id=c.contact_referring_account_id
),

contacts_final as (
    select 
        contact_id,
        account_id,
        contact_referring_contact_id,
        contact_referring_account_id,
        contact_owner_id,
        contact_name,
        contact_status,
        account_name as contact_account_name,
        contact_owner_name,
        contact_referring_contact_name,
        contact_referring_account_name
    from contacts_add_account
)

select * from contacts_final


## One more time with feeling
#### All Contact/Maple Syrup Campaigns

In [ ]:
-- collect everything we're going to need with an account.
with
accounts as (
    select *
    from staging.intermediate.int_salesforce__account
),

contacts as (
    select 
        id as contact_id,
        account_id as contact_account_id,
        name as contact_name,
        contact_status_c as contact_status,
        referred_by_c as contact_referred_by,
        contact_owner_id -- *    
    from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

get_columns as (
    select
        a.id as account_id,
        a.name as account_name,
        a.industry as account_industry,
        a.account_owner_id,
        a.account_owner_name,
        a.record_type_name as account_record_type_name,
        a.record_type_sobject_type as account_sobject_type_name,
        a.priority_c as account_priority,
        a.fund_of_fund_c as account_fund_of_fund,
        a.saas_non_saas_c as account_saas_non_saas,
        a.vc_strategy_c as account_vc_strategy,
        a.referring_entity_c as account_referring_account_id,
        a.referring_entity_contact_c as account_referring_contact_id,
        c.contact_name as account_referring_contact_name,
        a.key_intermediary_c as account_key_intermediary,
        date(a.created_date) as account_created_date
    from accounts a
    left join contacts c
        on c.contact_id=a.referring_entity_contact_c
),

final as (
    select 
        a.*,
        r.account_name as account_referring_entity_name
    from get_columns a
    left join get_columns r
        on a.account_referring_account_id=r.account_id
)

select * from final
where account_referring_contact_id is not null

In [ ]:
-- gather all necessary data for opportunity
with
opps as (
    select *
    from staging.intermediate.int_salesforce__opportunity
),

get_columns as (
    select
        id as opportunity_id,
        name as opportunity_name,
        account_id,
        stage_name as opportunity_stage_name,
        type as opportunity_type,
        owner_id as opportunity_owner_id,
        opportunity_owner_name,
        contact_id as opportunity_contact_id,
        lead_type_c as opportunity_lead_type,
        referral_contact_c as opportunity_referring_contact_id,
        referral_source_c as opportunity_referring_account_id,
        date(created_date) as opportunity_created_date
    from opps
)

select * from get_columns

# Step 2
## Show Opportunities and Their Current Stage

In [ ]:
-- get latest opportunity stage
with
opp as (
    select * from staging.intermediate.int_salesforce__opportunity
),

opphist as (
    select *
    from staging.salesforce.stg_salesforce__opportunity_history
),

stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from opphist
),

current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from stages
    where rn=1
),

opp_create_date as (
    select 
        id as opportunity_id,
        date(created_date) as opp_created_date
    from opp
),

final as (
    select 
        s.opportunity_id,
        s.current_stage,
        s.stage_created_date,
        o.opp_created_date
    from current_stage s
    inner join opp_create_date o on o.opportunity_id=s.opportunity_id
)

select * from final

In [ ]:
-- get the full opportunity output
with
opps as (
    select *
    from staging.intermediate.int_salesforce__opportunity
),

opphist as (
    select *
    from staging.salesforce.stg_salesforce__opportunity_history
),

accounts as (
    select 
        id as account_id,
        name as account_name
    from staging.intermediate.int_salesforce__account
),

contacts as (
    select 
        id as contact_id,
        name as contact_name
    from staging.intermediate.int_salesforce__contact
),

opp_columns as (
    select
        id as opportunity_id,
        name as opportunity_name,
        account_id,
        stage_name as opportunity_stage_name,
        type as opportunity_type,
        owner_id as opportunity_owner_id,
        opportunity_owner_name,
        contact_id as opportunity_contact_id,
        -- lead_type_c as opportunity_lead_type,
        referral_contact_c as opportunity_referring_contact_id,
        referral_source_c as opportunity_referring_account_id,
        date(created_date) as opportunity_created_date
    from opps
),

stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from opphist
),

current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from stages
    where rn=1
),

opps_merge as (
    select 
        --o.*,
        o.account_id,
        oa.account_name,
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_stage_name,
        o.opportunity_type,
        o.opportunity_owner_name,
        o.opportunity_contact_id,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_account_id,
        a.account_name as opportunity_referring_account_name,
        o.opportunity_created_date,
        a.account_name as opportunity_referring_account_name,
        c.contact_name as opportunity_referring_contact_name,
        coalesce(s.stage_created_date,o.opportunity_created_date) as stage_created_date,
        coalesce(s.current_stage,'Open') as current_stage
    from opp_columns o
    left join current_stage s 
        on s.opportunity_id=o.opportunity_id
    left join accounts oa
        on o.account_id=oa.account_id
    left join accounts a
        on o.opportunity_referring_account_id=a.account_id
    left join contacts c
        on o.opportunity_referring_contact_id=c.contact_id
)

select * from opps_merge

# Step 3
## Get Account and Referring Account

In [ ]:
with
acc as (
    select * from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

get_acc_columns as (
    select 
        id as account_id,
        name as account_name,
        -- other columns
        referring_entity_c as account_referring_account,
        referring_entity_contact_c as account_referring_contact_id
    from acc
    where is_deleted=false
),

cont as (
    select
        id as contact_id,
        account_id as contact_account_id
    from staging.intermediate.int_salesforce__contact
),

orphan_acc as (
    select
        a.account_id,
        a.account_name,
        coalesce(a.account_referring_account,c.contact_account_id) as account_referring_account,
        a.account_referring_contact_id
    from get_acc_columns a
    left join cont c on c.contact_id=a.account_referring_contact_id
)


select * from orphan_acc
-- where account_referring_contact_id is not null

# Step 4
## Align Opportunity with Referring Account and Referring Contact

In [ ]:
with 
opp as (
    select * from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

acc as (
    select * from staging.intermediate.int_salesforce__account
),

cont as (
    select
        id as contact_id,
        account_id as contact_account_id
    from staging.intermediate.int_salesforce__contact
),

get_acc_columns as (
    select 
        id as account_id,
        name as account_name,
        referring_entity_c as acc_referring_account,
        referring_entity_contact_c as acc_referring_contact
    from acc
    where is_deleted=false
),

get_opp_columns as (
    select 
        o.id as opportunity,
        o.account_id as opp_account,
        o.contact_id as opp_contact,
        o.referral_contact_c as opp_referring_contact,
        -- o.referral_source_c as opp_referring_account,
        coalesce(o.referral_source_c,c.contact_account_id) as opp_referring_account
        -- c.contact_id
    from opp o
    left join cont c 
        on c.contact_id=o.referral_contact_c
    where o.is_deleted=false
)

select * from get_opp_columns
-- where opp_referring_contact is not null


# Final Step
## Pulling it all Together

In [ ]:
-- CAPAIGN ALL
with
campaigns as (
    select 
        id as campaign_id,
        name as campaign_name,
        'Yes' as is_maple_syrup,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from staging.salesforce.stg_salesforce__campaign
    -- doesn't support (?i) syntax :(
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id        
    from staging.salesforce.stg_salesforce__campaign_member
    where is_deleted=false
),

total_members as (
    select 
        campaign_id,
        count(distinct contact_id) as total_campaign_members
    from members
    group by campaign_id
),

full_campaigns as (
    select 
        c.*,
        t.total_campaign_members,
        m.contact_id
    from campaigns c
    inner join total_members t
        on t.campaign_id=c.campaign_id
    inner join members m 
        on m.campaign_id=c.campaign_id
),

contacts as (
    select 
        id as contact_id,
        account_id as contact_account_id,
        id || account_id as contact_key,
        name as contact_name,
        contact_status_c as contact_status,
        contact_owner_id 
    from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

final as (
    -- want to get all contacts, and flag those that are part of the maple campaigns.
    select 
        c.contact_key as contact_account_key,
        c.contact_key as contact_referred_account_key,
        c.contact_id,
        c.contact_name,
        c.contact_account_id,
        c.contact_status,
        c.contact_owner_id,
        f.campaign_id,
        f.campaign_name,
        f.campaign_start_date,
        f.campaign_end_date
    from contacts c 
    left join full_campaigns f
        on c.contact_id=f.contact_id
)

select * from final
where campaign_id is not null
order by 
    campaign_start_date, contact_id 


In [ ]:
with
src_campaigns as (
    select * from staging.salesforce.stg_salesforce__campaign
    where is_deleted=false
),

src_members as (
    select * from staging.salesforce.stg_salesforce__campaign_member
),

src_contacts as (
    select * from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

src_opportunities as (
    select * from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

src_accounts as (
    select * from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

src_opportunities_histories as (
    select * from staging.salesforce.stg_salesforce__opportunity_history
    where is_deleted=false
),

campaigns as (
    -- still only looking at Maple Syrup campaigns ... for now.
    select 
        id as campaign_id,
        name as campaign_name,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from src_campaigns
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id   
    from src_members
    where is_deleted=false
),

final_members as (
    select 
        c.campaign_id,
        c.campaign_name,
        c.campaign_start_date,
        c.campaign_end_date,
        m.contact_id as campaign_member_contact_id,
        ct.name as campaign_member_contact_name,
        ct.account_id as campaign_member_account_id,
        a.name as campaign_member_account_name
    from campaigns c
    inner join members m 
        on m.campaign_id=c.campaign_id
    left join src_contacts ct
        on ct.id=m.contact_id
    left join src_accounts a
        on a.id=ct.account_id
),

opp_stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from src_opportunities_histories
),

opp_current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from opp_stages
    where rn=1
),

opp_create_date as (
    select 
        id as opportunity_id,
        date(created_date) as opportunity_created_date,
        account_id as opportunity_account_id,
        name as opportunity_name,
        referral_source_c as opportunity_referring_account_id,
        referral_contact_c as opportunity_referring_contact_id  
    from src_opportunities
),

opp_add_referal_contacts as (
    select 
        s.opportunity_id,
        o.opportunity_name,
        s.current_stage as opportunity_current_stage,
        s.stage_created_date as opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        coalesce(o.opportunity_referring_account_id,c.account_id) as opportunity_referring_account_id,
        o.opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name
    from opp_current_stage s
    inner join opp_create_date o 
        on o.opportunity_id=s.opportunity_id
    left join src_contacts c
        on c.id=o.opportunity_referring_contact_id
),

final_opportunities as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        a.name as opportunity_account_name,
        o.opportunity_referring_account_id,
        ra.name as opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name
    from opp_add_referal_contacts o
    inner join src_accounts a
        on a.id=o.opportunity_account_id
    left join src_accounts ra
        on ra.id=o.opportunity_referring_account_id
),

pre_final as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        case
            when m0.campaign_member_contact_id is not null then 'direct' else 'none'
        end as member_relationship_to_opp
    from final_opportunities o
    left join final_members m0
        on m0.campaign_member_contact_id=opportunity_referring_contact_id
        
    union all
    
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        'indirect' as member_relationship_to_opp
    from final_opportunities o
    inner join final_members m0
        on (m0.campaign_member_account_id=opportunity_referring_account_id and m0.campaign_member_contact_id<>opportunity_referring_contact_id)
),

final as ( 
    select
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy
    from pre_final
)

select * from final 
where opportunity_referring_account_id is not null
  and campaign_member_contact_name='Steven Abrams'

In [ ]:
-- CAPAIGN ALL
with
-- define sources
src_campaigns as (
    select * from staging.salesforce.stg_salesforce__campaign
    where is_deleted=false
),

src_members as (
    select * from staging.salesforce.stg_salesforce__campaign_member
),

src_contacts as (
    select * from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

src_opportunities as (
    select * from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
),

src_accounts as (
    select * from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

src_opportunities_histories as (
    select * from staging.salesforce.stg_salesforce__opportunity_history
    where is_deleted=false
),

campaigns as (
    -- still only looking at Maple Syrup campaigns ... for now.
    select 
        id as campaign_id,
        name as campaign_name,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from src_campaigns
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id   
    from src_members
    where is_deleted=false
),

final_members as (
    select 
        c.campaign_id,
        c.campaign_name,
        c.campaign_start_date,
        c.campaign_end_date,
        m.contact_id as campaign_member_contact_id,
        ct.name as campaign_member_contact_name,
        ct.account_id as campaign_member_account_id,
        a.name as campaign_member_account_name
    from campaigns c
    inner join members m 
        on m.campaign_id=c.campaign_id
    left join src_contacts ct
        on ct.id=m.contact_id
    left join src_accounts a
        on a.id=ct.account_id
),

-- validation for final_members
-- select * from final_members

-- do the account stuff here.
acc_get_columns as (
    select 
        id as account_id,
        name as account_name,
        -- other columns
        referring_entity_c as account_referring_account,
        referring_entity_contact_c as account_referring_contact_id
    from src_accounts
    where is_deleted=false
),

acc_contacts as (
    select
        id as contact_id,
        account_id as contact_account_id
    from src_contacts
),

acc_orphan_acc as (
    select
        a.account_id,
        a.account_name,
        coalesce(a.account_referring_account,c.contact_account_id) as account_referring_account_id,
        a.account_referring_contact_id
    from acc_get_columns a
    left join acc_contacts c 
        on c.contact_id=a.account_referring_contact_id
),

final_accounts as (
    select
        a.account_id,
        a.account_name,
        a.account_referring_account_id,
        ar.account_name as account_referring_account_name,
        a.account_referring_contact_id,
        c.name as account_referring_contact_name
    from acc_orphan_acc a
    left join acc_get_columns ar
        on ar.account_id=a.account_referring_account_id
    left join src_contacts c
        on c.id=a.account_referring_contact_id
    
),

-- select * from final_accounts

-- add opportunities now
-- starting with all opps and stages.
opp_stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from src_opportunities_histories
),

opp_current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from opp_stages
    where rn=1
),

opp_create_date as (
    select 
        id as opportunity_id,
        date(created_date) as opportunity_created_date,
        account_id as opportunity_account_id,
        name as opportunity_name,
        referral_source_c as opportunity_referring_account_id,
        referral_contact_c as opportunity_referring_contact_id  
    from src_opportunities
),

opp_add_referal_contacts as (
    select 
        s.opportunity_id,
        o.opportunity_name,
        s.current_stage as opportunity_current_stage,
        s.stage_created_date as opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        coalesce(o.opportunity_referring_account_id,c.account_id) as opportunity_referring_account_id,
        o.opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name
    from opp_current_stage s
    inner join opp_create_date o 
        on o.opportunity_id=s.opportunity_id
    left join src_contacts c
        on c.id=o.opportunity_referring_contact_id
),

final_opportunities as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        a.name as opportunity_account_name,
        o.opportunity_referring_account_id,
        ra.name as opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name
    from opp_add_referal_contacts o
    inner join src_accounts a
        on a.id=o.opportunity_account_id
    left join src_accounts ra
        on ra.id=o.opportunity_referring_account_id
),

-- select * from final_opportunities
-- where opportunity_referring_contact_id is not null

-- I have 3 final tables that need to be brought together for one final table
-- final_opportunities
-- final_members
-- final_accounts 

direct as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        case
            when m0.campaign_member_contact_id is not null then 'direct' else 'none'
        end as member_relationship_to_opp
    from final_opportunities o
    inner join final_members m0
        on m0.campaign_member_contact_id=opportunity_referring_contact_id
),

indirect as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        c.id as opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        'indirect' as member_relationship_to_opp
    from final_opportunities o
    inner join final_members m0
        on (m0.campaign_member_account_id=opportunity_referring_account_id and m0.campaign_member_contact_id<>o.opportunity_referring_contact_id)
    left join src_contacts c
        on c.account_id=o.opportunity_referring_account_id
    where (o.opportunity_id, m0.campaign_id, c.id) 
        not in (select opportunity_id, campaign_id, campaign_member_contact_id from direct)
),

no_campaign as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        c.id as opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name,
        null as campaign_id,
        null as campaign_name,
        null as campaign_start_date,
        null as campaign_end_date,
        c.id as campaign_member_contact_id,
        c.name as campaign_member_contact_name,
        'none' as member_relationship_to_opp
    from final_opportunities o
    inner join src_contacts c
        on c.account_id=o.opportunity_referring_account_id -- and c.id<>o.opportunity_referring_contact_id
    where o.opportunity_id not in (select opportunity_id from direct)
),

-- pre_final as (
--     select 
--         o.opportunity_id,
--         o.opportunity_name,
--         o.opportunity_current_stage,
--         o.opportunity_stage_created_date,
--         o.opportunity_created_date,
--         o.opportunity_account_id,
--         o.opportunity_account_name,
--         o.opportunity_referring_account_id,
--         o.opportunity_referring_account_name,
--         o.opportunity_referring_contact_id,
--         o.opportunity_referring_contact_name,
--         m0.campaign_id,
--         m0.campaign_name,
--         m0.campaign_start_date,
--         m0.campaign_end_date,
--         m0.campaign_member_contact_id,
--         m0.campaign_member_contact_name,
--         case
--             when m0.campaign_member_contact_id is not null then 'direct' else 'none'
--         end as member_relationship_to_opp
--     from final_opportunities o
--     inner join final_members m0
--         on m0.campaign_member_contact_id=opportunity_referring_contact_id
        
--     union all

--     select 
--         o.opportunity_id,
--         o.opportunity_name,
--         o.opportunity_current_stage,
--         o.opportunity_stage_created_date,
--         o.opportunity_created_date,
--         o.opportunity_account_id,
--         o.opportunity_account_name,
--         o.opportunity_referring_account_id,
--         o.opportunity_referring_account_name,
--         c.id as opportunity_referring_contact_id,
--         c.name as opportunity_referring_contact_name,
--         m0.campaign_id,
--         m0.campaign_name,
--         m0.campaign_start_date,
--         m0.campaign_end_date,
--         m0.campaign_member_contact_id,
--         m0.campaign_member_contact_name,
--         'indirect' as member_relationship_to_opp
--     from final_opportunities o
--     inner join final_members m0
--         on (m0.campaign_member_account_id=opportunity_referring_account_id and m0.campaign_member_contact_id<>o.opportunity_referring_contact_id)
--     left join src_contacts c
--         on c.account_id=o.opportunity_referring_account_id

--     union all

--     select 
--         o.opportunity_id,
--         o.opportunity_name,
--         o.opportunity_current_stage,
--         o.opportunity_stage_created_date,
--         o.opportunity_created_date,
--         o.opportunity_account_id,
--         o.opportunity_account_name,
--         o.opportunity_referring_account_id,
--         o.opportunity_referring_account_name,
--         c.id as opportunity_referring_contact_id,
--         c.name as opportunity_referring_contact_name,
--         null as campaign_id,
--         null as campaign_name,
--         null as campaign_start_date,
--         null as campaign_end_date,
--         c.id as campaign_member_contact_id,
--         c.name as campaign_member_contact_name,
--         'none' as member_relationship_to_opp
--     from final_opportunities o
--     inner join src_contacts c
--         on c.account_id=o.opportunity_referring_account_id and c.id<>o.opportunity_referring_contact_id
-- ),

final as (
    select
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy
    from no_campaign
)

select * from final 
-- where opportunity_referring_account_id is not null
--   and opportunity_referring_account_name='BDC Capital'
  -- and opportunity
  

# Final with Changs

In [ ]:
with
src_campaigns as (
    select * from staging.salesforce.stg_salesforce__campaign
    where is_deleted=false
),

src_members as (
    select * from staging.salesforce.stg_salesforce__campaign_member
),

src_contacts as (
    select * from staging.intermediate.int_salesforce__contact
    where is_deleted=false
),

src_opportunities as (
    select * from staging.intermediate.int_salesforce__opportunity
    where is_deleted=false
      and coalesce(sred_deal_c,true)=false
      and name not ilike '%sred%'
),

src_accounts as (
    select * from staging.intermediate.int_salesforce__account
    where is_deleted=false
),

src_opportunities_histories as (
    select * from staging.salesforce.stg_salesforce__opportunity_history
    where is_deleted=false
),

campaigns as (
    select 
        id as campaign_id,
        name as campaign_name,
        start_date as campaign_start_date,
        end_date as campaign_end_date
    from src_campaigns
    where name rlike '^[0-9]{4}\\s+.*\\b[Mm][Aa][Pp][Ll][Ee]\\b.*\\b[Ss][Yy][Rr][Uu][Pp]\\b.*$'
      and is_deleted=false
),

members as (
    select 
        campaign_id,
        contact_id   
    from src_members
    where is_deleted=false
),

final_members as (
    select 
        c.campaign_id,
        c.campaign_name,
        c.campaign_start_date,
        c.campaign_end_date,
        m.contact_id as campaign_member_contact_id,
        ct.name as campaign_member_contact_name,
        ct.account_id as campaign_member_account_id,
        a.name as campaign_member_account_name
    from campaigns c
    inner join members m 
        on m.campaign_id=c.campaign_id
    left join src_contacts ct
        on ct.id=m.contact_id
    left join src_accounts a
        on a.id=ct.account_id
),

acc_get_columns as (
    select 
        id as account_id,
        name as account_name,
        -- other columns
        referring_entity_c as account_referring_account,
        referring_entity_contact_c as account_referring_contact_id
    from src_accounts
    where is_deleted=false
),

acc_contacts as (
    select
        id as contact_id,
        account_id as contact_account_id
    from src_contacts
),

acc_orphan_acc as (
    select
        a.account_id,
        a.account_name,
        coalesce(a.account_referring_account,c.contact_account_id) as account_referring_account_id,
        a.account_referring_contact_id
    from acc_get_columns a
    left join acc_contacts c 
        on c.contact_id=a.account_referring_contact_id
),

final_accounts as (
    select
        a.account_id,
        a.account_name,
        a.account_referring_account_id,
        ar.account_name as account_referring_account_name,
        a.account_referring_contact_id,
        c.name as account_referring_contact_name
    from acc_orphan_acc a
    left join acc_get_columns ar
        on ar.account_id=a.account_referring_account_id
    left join src_contacts c
        on c.id=a.account_referring_contact_id
    
),

opp_stages as (
    select
        opportunity_id,
        stage_name,
        date(created_date) as stage_created_date,
        row_number() over (
            partition by opportunity_id
            order by created_date desc
        ) as rn
    from src_opportunities_histories
),

opp_current_stage as (
    select 
        opportunity_id,
        stage_name as current_stage,
        stage_created_date
    from opp_stages
    where rn=1
),

opp_create_date as (
    select 
        id as opportunity_id,
        date(created_date) as opportunity_created_date,
        account_id as opportunity_account_id,
        name as opportunity_name,
        referral_source_c as opportunity_referring_account_id,
        referral_contact_c as opportunity_referring_contact_id  
    from src_opportunities
),

opp_add_referal_contacts as (
    select 
        s.opportunity_id,
        o.opportunity_name,
        s.current_stage as opportunity_current_stage,
        s.stage_created_date as opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        coalesce(o.opportunity_referring_account_id,c.account_id) as opportunity_referring_account_id,
        o.opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name
    from opp_current_stage s
    inner join opp_create_date o 
        on o.opportunity_id=s.opportunity_id
    left join src_contacts c
        on c.id=o.opportunity_referring_contact_id
),

final_opportunities as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        a.name as opportunity_account_name,
        o.opportunity_referring_account_id,
        ra.name as opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name
    from opp_add_referal_contacts o
    inner join src_accounts a
        on a.id=o.opportunity_account_id
    left join src_accounts ra
        on ra.id=o.opportunity_referring_account_id
),

direct as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        o.opportunity_referring_contact_id,
        o.opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        case
            when m0.campaign_member_contact_id is not null then 'direct' else 'none'
        end as member_relationship_to_opp
    from final_opportunities o
    inner join final_members m0
        on m0.campaign_member_contact_id=opportunity_referring_contact_id
),

indirect as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        c.id as opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name,
        m0.campaign_id,
        m0.campaign_name,
        m0.campaign_start_date,
        m0.campaign_end_date,
        m0.campaign_member_contact_id,
        m0.campaign_member_contact_name,
        'indirect' as member_relationship_to_opp
    from final_opportunities o
    inner join final_members m0
        on (m0.campaign_member_account_id=opportunity_referring_account_id and m0.campaign_member_contact_id<>o.opportunity_referring_contact_id)
    left join src_contacts c
        on c.account_id=o.opportunity_referring_account_id
    where (o.opportunity_id, m0.campaign_id, c.id) 
        not in (select opportunity_id, campaign_id, campaign_member_contact_id from direct)
),

no_campaign as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        o.opportunity_referring_account_id,
        o.opportunity_referring_account_name,
        c.id as opportunity_referring_contact_id,
        c.name as opportunity_referring_contact_name,
        null as campaign_id,
        null as campaign_name,
        null as campaign_start_date,
        null as campaign_end_date,
        c.id as campaign_member_contact_id,
        c.name as campaign_member_contact_name,
        'none' as member_relationship_to_opp
    from final_opportunities o
    inner join src_contacts c
        on c.account_id=o.opportunity_referring_account_id 
    where o.opportunity_id not in (select opportunity_id from direct)
),

no_referral as (
    select 
        o.opportunity_id,
        o.opportunity_name,
        o.opportunity_current_stage,
        o.opportunity_stage_created_date,
        o.opportunity_created_date,
        o.opportunity_account_id,
        o.opportunity_account_name,
        null as opportunity_referring_account_id,
        null as opportunity_referring_account_name,
        null as opportunity_referring_contact_id,
        null as opportunity_referring_contact_name,
        null as campaign_id,
        null as campaign_name,
        null as campaign_start_date,
        null as campaign_end_date,
        null as campaign_member_contact_id,
        null as campaign_member_contact_name,
        'no_referral' as member_relationship_to_opp
    from final_opportunities o
    where o.opportunity_referring_account_id is null
),

final as (
    select 
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy     
    from direct
    union all
    select 
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy     
    from indirect
    union all
    select 
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy     
    from no_campaign    
    union all
    select 
        *,
        to_char(opportunity_created_date, 'YYYY-MM') as opportunity_created_yyyy_mm,
        to_char(opportunity_created_date, 'YYYY') as opportunity_created_yyyy     
    from no_referral
)

select count(distinct opportunity_id) from final

-- select count(distinct id) from src_opportunities --final 
-- where opportunity_referring_account_id is null
--   and opportunity_referring_account_name='BDC Capital'
  -- and opportunity
  